In [ ]:
!pip install -q openai

In [ ]:
# from openai import OpenAI
import openai

In [ ]:
openai.api_key = 'your api_key'

## intents and slots

Based on dataset

In [ ]:
# Facebook Multilingual
intents_list = ['alarm/set_alarm', 'alarm/show_alarms', 'alarm/cancel_alarm', 'alarm/time_left_on_alarm', 'alarm/modify_alarm', 'alarm/snooze_alarm', 'reminder/set_reminder', 'reminder/show_reminders', 'reminder/cancel_reminder', 'weather/find', 'weather/checkSunrise', 'weather/checkSunset']

In [ ]:
# Persian ATIS
intents_list = [
'flight_no',
'airfare+flight',
'capacity',
'airfare+flight_time',
'quantity',
'airfare',
'ground_service+ground_fare',
'city',
'flight_no+airline',
'flight',
'flight+airfare',
'airport',
'abbreviation',
'cheapest',
'aircraft+flight+flight_no',
'distance',
'restriction',
'meal',
'aircraft',
'flight_time',
'flight+airline',
'ground_fare',
'airline+flight_no',
'airline',
'ground_service',
'day_name'
]

In [ ]:
# Facebook Multilingual
slots_list = ['datetime', 'alarm/alarm_modifier', 'reminder/noun', 'reminder/todo',
              'reminder/reference', 'reminder/reminder_modifier', 'reminder/recurring_period', 'location',
              'weather/temperatureUnit', 'weather/noun', 'weather/attribute']

In [ ]:
# Persian ATIS
slots_list = ['cost_relative',
 'class_type',
 'toloc.state_code',
 'fromloc.city_name',
 'toloc.state_name',
 'return_date.today_relative',
 'fare_amount',
 'meal',
 'return_date.day_name',
 'restriction_code',
 'flight_time',
 'arrive_time.time',
 'depart_time.start_time',
 'fromloc.airport_name',
 'depart_date.day_number',
 'fare_basis_code',
 'depart_date.today_relative',
 'return_date.date_relative',
 'toloc.country_name',
 'depart_time.period_mod',
 'toloc.airport_name',
 'day_name',
 'arrive_time.period_of_day',
 'toloc.airport_code',
 'depart_time.period_of_day',
 'days_code',
 'return_time.period_of_day',
 'arrive_time.end_time',
 'time',
 'toloc.city_name',
 'arrive_time.time_relative',
 'state_name',
 'depart_date.day_name',
 'fromloc.state_code',
 'flight_number',
 'fromloc.state_name',
 'flight',
 'arrive_date.day_number',
 'depart_time.time',
 'airport_name',
 'stoploc.state_code',
 'depart_time.time_relative',
 'meal_description',
 'round_trip',
 'return_time.period_mod',
 'arrive_time.start_time',
 'state_code',
 'fromloc.airport_code',
 'return_date.month_name',
 'stoploc.city_name',
 'flight_stop',
 'airline_name',
 'O',
 'depart_time.end_time',
 'compartment',
 'period_of_day',
 'transport_type',
 'stoploc.airport_name',
 'today_relative',
 'mod',
 'booking_class',
 'arrive_date.date_relative',
 'airport_code',
 'economy',
 'meal_code',
 'depart_date.date_relative',
 'arrive_time.period_mod',
 'time_relative',
 'arrive_date.today_relative',
 'depart_date.year',
 'day_number',
 'flight_mod',
 'depart_date.month_name',
 'city_name',
 'return_date.day_number',
 'arrive_date.day_name',
 'airline_code',
 'connect',
 'or',
 'flight_days',
 'aircraft_code',
 'month_name',
 'arrive_date.month_name',
 'stoploc.airport_code']

# Promts

**ADD EXAMPLES FOR FEW-SHOT PROMPTING**

## ID

In [ ]:
template_format = (
  "Here are the possible intents: \n"
  "{intents}\n\n"
  "I will provide you with an input utterance. \n"
  "Your task is to identify the intent of the input text by selecting the most relevant intent from the list. \n\n"
  "Now, process this utterance (only write the intent):\n\n"
  "{input}"
)

In [ ]:
# # Example usage
# input_utterance = "input1"  # Replace this with the actual input
# prompt = template_format.format(
#     intents=intents_list,
#     input=input_utterance
# )
# print(prompt)

## SF

In [ ]:
template_prompt = (
    "Here are the possible slots: \n"
    "{slots}\n\n"
    "I will provide you with an input utterance. \n"
     "Your task is to extract and return the slot values in a structured JSON format using BIO tags for slot filling. \n\n"
    "The output structure should be:\n"
    "{{\n"
    "    \"slots\": [\n"
    "        {{\"token\": \"word1\", \"label\": \"B-slot_name\"}},\n"
    "        {{\"token\": \"word2\", \"label\": \"I-slot_name\"}},\n"
    "        ...\n"
    "    ]\n"
    "}}\n\n"
    "Now, process this utterance:\n\n"
    "{input}"
)

## Do it

In [ ]:
# File paths
input_file_path = "input.txt"
output_file_path = "output.txt"

In [ ]:
# System message to set initial assistant behavior
messages = [{"role": "system", "content": "You are a language model trained to perform intent detection."}]
# messages = [{"role": "system", "content": "You are a language model trained to perform slot filling."}] # uncomment for SF

c = 0
# Read from input file and write responses to output file
with open(input_file_path, 'r') as input_file, open(output_file_path, 'w') as output_file:
    for line in input_file:
        print(c)
        c = c+1
        # Strip newline characters and substitute into the template
        input_utterance = line.strip()
        if input_utterance:  # Only proceed if the line isn't empty

            prompt = template_format.format(
                intents=intents_list,
                input=input_utterance
            )

            # prompt = template_prompt.format(
            #     slots=slots_list,
            #     input=input_utterance
            # ) # uncomment for SF

            # Add user message to the conversation
            messages.append({"role": "user", "content": prompt})

            # Generate a response from GPT-4
            chat =  openai.chat.completions.create(
                model="gpt-4o",
                # temperature=0.2,
                # top_p=0.9,
                messages=messages
            ) # default values

            # Get and print the assistant's response
            reply = chat.choices[0].message.content
            print(f"ChatGPT: {reply}")
            print('-' * 60)

            # Write the response to the output file
            output_file.write(f"{reply}\n")

            # Add assistant's response to the conversation
            messages.append({"role": "assistant", "content": reply})

            # Clear messages for the next input (if each input is standalone)
            messages = [{"role": "system", "content": "You are a language model trained to perform intent detection."}]
            # messages = [{"role": "system", "content": "You are a language model trained to perform slot filling."}] # uncomment for SF

metric (Acc)

In [ ]:
out = open('path_to_gold_data', 'r')
gold_intent = out.readlines()
out.close()

In [ ]:
gold_intent = [intent.strip() for intent in gold_intent]

In [ ]:
# Read the file
with open('output.txt', 'r') as file:
    intents = file.readlines()

# Clean each line by removing surrounding quotes or spaces
cleaned_intents = [intent.strip().replace("'", "") for intent in intents]

test_intent = []
for intent in cleaned_intents:
    test_intent.append(intent)

In [ ]:
# Calculate accuracy
correct = sum(1 for g, o in zip(gold_intent, test_intent) if g == o)
accuracy = correct / len(gold_intent)

print(f"Accuracy: {accuracy:.4f}")

In [ ]:
# # Find incorrect indexes
# incorrect_indexes = [i for i, (g, o) in enumerate(zip(gold_intent, test_intent)) if g != o]

# # Print the incorrect indexes
# print("Incorrect indexes:", incorrect_indexes)

Extract for SF

In [ ]:
import json

# Function to extract labels from JSON strings
def extract_labels_from_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()

    # Split content based on the appearance of "json"
    parts = content.split('json')

    # Prepare a list to hold the labels
    labels_list = []
    c = 0
    for part in parts:
        # Find the first { and last } to capture the JSON object
        json_start = part.find('{')
        json_end = part.rfind('}')

        if json_start != -1 and json_end != -1:
            json_str = part[json_start:json_end+1]  # Extract the JSON block

            try:
                # Load JSON string as a dictionary
                data = json.loads(json_str)

                # Extract labels
                if 'slots' in data:
                    labels = [slot['label'] for slot in data['slots']]
                    labels_list.append(labels)
                    c = c+1
                    print(c)
                else:
                    print(f"Skipping JSON without 'slots': {json_str}")
            except json.JSONDecodeError:
                print(f"Error decoding JSON: {json_str}")

    return labels_list

# Function to write the labels to a file
def write_labels_to_file(labels, output_file):
    with open(output_file, 'w', encoding='utf-8') as file:
        for label_list in labels:
            # Convert list to string format and write to file
            file.write(f"{label_list}\n")

    file.close()

In [ ]:
labels = extract_labels_from_json('output.txt')

In [ ]:
output_file = 'output_L.txt'
write_labels_to_file(labels, output_file)
print(f"Labels have been written to {output_file}.")

In [ ]:
import ast

def read_extract(input_file):
  LL = []
  with open(input_file, 'r') as infile:
      for line in infile:
          # Use ast.literal_eval to safely evaluate the string as a list
          label_list = ast.literal_eval(line.strip())
          LL.append(label_list)
  return LL

In [ ]:
test_slot = read_extract('output_L.txt')
gold_slot = read_extract('path_to_gold_data')

In [ ]:
n=500
for i in range(n):
  if len(test_slot[i]) != len(gold_slot[i]):
    print(i) # find specific_indices

In [ ]:
# #align
for a in [specific_indices]:
  if len(test_slot[a]) > len(gold_slot[a]):
      # Truncate test to match the length of gold
      test_slot[a] = test_slot[a][:len(gold_slot[a])]
  elif len(test_slot[a]) < len(gold_slot[a]):
      # Pad test with "O" to match the length of gold
      test_slot[a].extend(["O"] * (len(gold_slot[a]) - len(test_slot[a])))

metric (F1)

In [ ]:
!pip install seqeval

In [ ]:
from seqeval.metrics import classification_report

In [ ]:
print(classification_report(gold_slot, test_slot))